# Synthetic Data Generation for Named Entity Recognition Benchmarking

## Overview
This notebook generates diverse synthetic data examples to benchmark multiple open-source models for Named Entity Recognition (NER) tasks. For the purpose of donor relation email parsing.


In [ ]:
faculties = {
    "Faculty of Agricultural and Food Sciences",
    "School of Agriculture",
    "School of Art",
    "Faculty of Arts",
    "I. H. Asper School of Business",
    "Faculty of Education",
    "Price Faculty of Engineering",
    "Clayton H. Riddell Faculty of Environment, Earth, and Resources",
    "Extended Education",
    "Faculty of Graduate Studies",
    "Rady Faculty of Health Sciences",
    "School of Dental Hygiene",
    "Dr. Gerald Niznick College of Dentistry",
    "Max Rady College of Medicine",
    "College of Nursing",
    "College of Pharmacy",
    "College of Rehabilitation Sciences",
    "Faculty of Kinesiology and Recreation Management",
    "Faculty of Law",
    "Desautels Faculty of Music",
    "Faculty of Science",
    "Faculty of Social Work",
    "University 1"
}

programs = {
    "Accounting",
    "Actuarial Mathematics - Business",
    "Actuarial Mathematics - Science",
    "Aging (interfaculty option)",
    "Agribusiness",
    "Agriculture",
    "Agriculture Diploma",
    "Agroecology",
    "Agronomy",
    "Animal Systems",
    "Anthropology",
    "Applied Mathematics",
    "Architecture (Masters)",
    "Art",
    "Art History (BA)",
    "Art History (BFA)",
    "Asian Studies",
    "Athletic Therapy",
    "Biochemistry",
    "Biological Sciences",
    "Biosystems Engineering",
    "Business",
    "Business Analytics (BComm) (Honours)",
    "Canadian Private Law (Micro-Diploma)",
    "Canadian Public Law (Micro-Diploma)",
    "Canadian Studies",
    "Catholic Studies",
    "Central and East European Studies",
    "Ceramics",
    "Chemistry",
    "City Planning (Masters)",
    "Civil Engineering",
    "Classics",
    "Commerce",
    "Computer Engineering",
    "Computer Science",
    "Criminology",
    "Data Science",
    "Dental Hygiene (BScDH)",
    "Dental Hygiene (Diploma)",
    "Dentistry (BSc)",
    "Dentistry (DMD)",
    "Dentistry (DMD/PhD)",
    "Drawing",
    "Earth Sciences",
    "Economics",
    "Education - Bachelor of Education",
    "Education - Post Baccalaureate",
    "Electrical Engineering",
    "Engineering",
    "English",
    "Entrepreneurship/Small Business",
    "Environmental Design",
    "Environmental Geoscience (BSc)",
    "Environmental Science",
    "Environmental Studies",
    "Essentials in Advanced Patient Care for Pharmacists (Micro-Certificate)",
    "Film Studies",
    "Finance",
    "Fine Arts",
    "Food Science",
    "French",
    "Genetics",
    "Geography",
    "Geology",
    "Geophysics",
    "German",
    "German Language, Life and Culture (Micro-Diploma)",
    "Global Political Economy",
    "Graphic Design",
    "Greek",
    "Health Sciences",
    "Health Studies",
    "History",
    "Human Nutritional Sciences",
    "Human Resource Management / Industrial Relations",
    "Icelandic",
    "Indigenous Business Studies",
    "Indigenous Governance",
    "Indigenous Languages",
    "Indigenous Studies",
    "Interior Design (Masters)",
    "International Business",
    "International Dentist Degree Program",
    "International Medical Graduate Program",
    "Italian",
    "Jazz Studies",
    "Judaic Studies",
    "Juris Doctor (JD)",
    "Labour Relations and Workplace Studies (Diploma)",
    "Labour Studies",
    "Landscape Architecture (Masters)",
    "Latin",
    "Latin American Studies",
    "Law",
    "Leadership and Organizations",
    "Leadership for Business and Organizations (Minor)",
    "Linguistics",
    "Logistics and Supply Chain Management",
    "Management",
    "Management Information Systems",
    "Management Minor for Non-Business Students",
    "Marketing",
    "Mathematics",
    "Mechanical Engineering",
    "Medicine",
    "Medicine (BSc Med)",
    "Medieval and Early Modern Studies",
    "Microbiology",
    "Midwifery",
    "Music",
    "Music (Post-Baccalaureate Diploma in Performance)",
    "Music Minor",
    "Mythology and Folktale (Micro-Diploma)",
    "Nursing",
    "Occupational Therapy (Masters)",
    "Painting",
    "Pharmacy",
    "Philosophy",
    "Photography",
    "Physical Education",
    "Physical Geography",
    "Physics and Astronomy",
    "Physiology and Pathophysiology (postbaccalaureate diploma)",
    "Plant Biotechnology",
    "Polish (Minor)",
    "Political Studies",
    "Pre-Veterinary Medicine",
    "Print Media",
    "Psychology - Arts",
    "Psychology - Science",
    "Recreation Management and Community Development",
    "Recreation Studies (Minor)",
    "Religion",
    "Respiratory Therapy",
    "Russian",
    "Science",
    "Sculpture",
    "Social Work – Distance Delivery",
    "Social Work – Fort Garry campus",
    "Social Work – Inner City Program at William Norrie Centre",
    "Social Work – Northern Program in Thompson",
    "Sociology",
    "Songmaking (Micro-Certificate)",
    "Spanish",
    "Sport, Physical Activity and Recreation in the Community Certificate (SPARC)",
    "Statistics",
    "Strategy and Global Management (BComm) (Honours)",
    "Strategy and Global Management",
    "Theatre",
    "Ukrainian",
    "Ukrainian Canadian Heritage Studies",
    "University 1",
    "Video",
    "Women's and Gender Studies",
    "Workplace Health and Safety (Micro-Diploma)"
}

colleges = {
    "St. Andrew’s College",
    "St. John’s College",
    "St. Paul’s College",
    "University College"
}

